In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

%matplotlib inline

import math
import numpy as np
import requests
import ipywidgets as widgets

import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

import parmed as pmd
import re
from scipy.ndimage import gaussian_filter
import time

import threading

import hublib.use
from hublib.ui import Submit

%use gromacs-2018.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)

In [ ]:
def writelog(msg, leadchar):
    with open("./logfile.txt", 'a') as fd:
        for line in msg:
            fd.write(leadchar + " " + line + "\n")
            
def runbash(cmd, comment):
    out = !{cmd}
    writelog(["*****************************************************"], '')
    writelog([comment], '# ')
    writelog(["Running bash command:"], '# ')
    writelog([cmd+'\n'], 'bash >> ')
    writelog(out, 'out: ')
    writelog(["*****************************************************"], '')
    return out

In [ ]:
def initial_setup():

    runbash('cp ../pdb2gmx/pdb2gmx.gro ./', 'Copying pdb2gmx.gro from pdb2gmx folder')
    runbash('cp ../pdb2gmx/complex.top ./', 'Copying complex.top from pdb2gmx folder')
    runbash('cp ../pdb2gmx/*.itp ./', 'Copying all *.itp inputs from pdb2gmx folder')

    # Use editconf to set the box size. 
    out = runbash("gmx editconf -f pdb2gmx.gro -o newbox.gro -bt cubic -d 2", "using gmx editconf to ensure 2 nm distance to each wall")

    # Compile a *.tpr file for the energy minimization
    out = runbash("gmx grompp -maxwarn 1 -f em.mdp -c newbox.gro -p complex.top -o em.tpr", "Compiling em.tpr binary for minimization")
    
    
def run_sim(s):
    
    if os.path.isfile('em.gro') and os.path.isfile('em.pdb') and os.path.isfile('charges.txt'):
        print('The simulation appears to have already completed. If you want to run it again, refresh the page and click Purge.')
        return
    
    elif os.path.isfile('em.tpr'):
        rname = 'em'
        writelog(["Running energy minimization for hydrogen atoms (restrained heavy atoms)", 
                  "Submitting job with command: submit -w 01:00:00 -v standby@brown -i hmin.tpr gromacs-2018.4-gmx mdrun -v -deffnm hmin"], '# ')
    
        substr = "-w 01:00:00 -v standby@brown -i em.tpr gromacs-2018.4-gmx mdrun -v -deffnm em"
        
        print("Submitting energy minimization (no restraints). ")
        print("Do NOT refresh or navigate away from this page until it completes (simulation will abort).")
        s.run(rname, substr)
        
    else:
        print('Could not find input file em.tpr. Aborting job submission.')
        

def on_finish(s, rdir):
    
    # Do the final cleanup of structure files
    final_rinse()
        
    # And plot the potential. 
    plot_potential()

    
def final_rinse():
    
    # Print the charges
    out = runbash("gmx editconf -f em.tpr -mead -o charge.pdb", "Preparing mead.pqr file for charge extraction.")
    out = runbash("bash parse_pqr.sh mead.pqr", "Printing charges to charges.txt")

    # Output a PDB file for viewing
    out = runbash("echo -e 0 | gmx trjconv -f em.gro -s em.tpr -o temp.pdb", "Using gmx trjconv to output temp.pdb for viewing.")
    writelog(["Associating with chain 0 all residues that have not yet been assigned a chain.",
                 "Final structure stored in em.pdb"], "# ")
    with open('em.pdb', 'w') as ofd:
        with open('temp.pdb') as ifd:
            for line in ifd:
                if len(line)>22:
                    if line[21]==' ':
                        line = line[:21] + '$' + line[22:]
                ofd.write(line)



def plot_potential():
    # Now we want to generate a potential energy curve for validation. 
    # But first we have to find out which index in gmx energy corresponds to the potential. 
    # Send a dummy command to gmx energy that will throw an error but let us see the 
    # options for output. 
    out = runbash("echo -e 0 | gmx energy -f em.edr", "Running dummy gmx energy command to check input index options")
    addtext = False
    SelText = ''
    # Parse through the output 
    for line in out:
        if line[0:10]=='----------':
            if addtext:
                break
            else:
                addtext = True
        else:
            if addtext:
                SelText += line + " "
    try: 
        # search the string for an occurance of "x Potential y" where x and y are nonnegative integers
        hit = re.match('.*[ \t]+([0-9]+[ \t]+)(Potential)[ \t]+[0-9]+.*', SelText)

        # The first group should be the number for the Potential Energy
        PotNum = hit.group(1).strip()

        writelog(["Parsed gmx energy output and itentified " + PotNum + " as index for potential energy"], "# ")

        # Now run the real potential energy command
        out = runbash("echo -e "+PotNum+" 0 | gmx energy -f em.edr -o potential.xvg",
                "Printing potential energy trajectory to file potential.xvg")

        # Extract title from plot.
        title = runbash("sed -n \'s/\(@ s0 legend \"\)\(.*\)\"/\\2/p\' potential.xvg", 'Extracting plot title')[0]
        yunits = runbash("sed -n \'s/\(@    yaxis  label \"\)\(.*\)\"/\\2/p\' potential.xvg", 'Extracting y-axis units')[0]
        xlabel = runbash("sed -n \'s/\(@    xaxis  label \"\)\(.*\)\"/\\2/p\' potential.xvg", 'Extracting x-axis units')[0]

        display_markdown('# ' + title + ' #', raw=True)
        display_markdown('Please check for consistency. You should see a smooth decay.', raw=True)
        fig = plt.figure(figsize=(6,4))

        dat = np.loadtxt('potential.xvg', comments=['@', '#'])

        plt.plot(dat[:,0], dat[:,1])
        plt.xlabel(xlabel, fontsize=16)
        plt.ylabel(title + ' ' + yunits, fontsize=16)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.tight_layout()
        plt.show()

    # If for some reason parsing fails, print an error and skip potential energy output. 
    except:
        print('Error identifying Potential energy index in gmx energy. Cannot print potential energy curve.')
        writelog(['Error identifying Potential energy index in gmx energy. Cannot print potential energy curve.'], '# ')


In [ ]:
################################################################
# Main sequence
################################################################

purge_bt = widgets.Button(description='Purge Files')
def purge_files(b):
    if os.path.isfile('em.gro'):
        runbash('rm em.gro', 'Deleting em.gro')
    if os.path.isfile('em.pdb'):
        runbash('rm em.pdb', 'Deleting em.pdb')
    if os.path.isfile('charges.txt'):
        runbash('rm charges.txt', 'Deleting charges.txt')

purge_bt.on_click(purge_files)

# If required outputs already exist, don't run automatically
if os.path.isfile('em.gro') and os.path.isfile('em.pdb') and os.path.isfile('charges.txt'):
    print('This script appears already to have run. If you want to run it again, click below to purge files and then refresh the page.')
    display(purge_bt)
    plot_potential()
    
else:
    initial_setup()
    
Submit(start_func=run_sim, done_func=on_finish)